In [6]:
import paddle
from data_process import get_MNIST_dataloader
from MNIST_network import MNIST

train_loader, test_loader = get_MNIST_dataloader()

e:\SoftWare\Program\Anaconda\envs\d2l\lib\site-packages\paddle\fluid\reader.py:486: UserWarning: DataLoader with multi-process mode is not supported on MacOs and Windows currently. Please use signle-process mode with num_workers = 0 instead
  warnings.warn(


In [2]:
import paddle

device = paddle.device.get_device()

print('device:', device)

device: gpu:0


In [3]:
import paddle
x = paddle.to_tensor(1.0, place=paddle.CPUPlace())
print(x.place)        # CPUPlace

y = x.cuda()
print(y.place)        # CUDAPlace(0)

Place(cpu)
Place(gpu:0)


In [7]:
import paddle.nn.functional as F

def train(model):
    # 开启GPU
    use_gpu = True
    paddle.device.set_device('gpu:0') if use_gpu else paddle.device.set_device('cpu')
    model.train()

    # 设置不同的学习率
    opt = paddle.optimizer.Adam(learning_rate=0.01, parameters=model.parameters())

    EPOCH_NUM = 5
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            images, labels = data 
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)

            # 前向
            predicts = model(images)

            # 损失
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)

            # 每训练100，打印
            if batch_id%100 == 0:
                print(f'epoch_id{epoch_id}, batch_id{batch_id}, loss{avg_loss}')
            
            # 损失函数的方向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()
    
    # 保存参数
    paddle.save(model.state_dict(),'mnist.pdparams')

model = MNIST()

train(model)

epoch_id0, batch_id0, lossTensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [5.00350571])
epoch_id0, batch_id100, lossTensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [2.29112077])
epoch_id0, batch_id200, lossTensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [2.30266857])
epoch_id0, batch_id300, lossTensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [2.32182217])
epoch_id0, batch_id400, lossTensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [2.31923318])
epoch_id0, batch_id500, lossTensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [2.29653788])
epoch_id0, batch_id600, lossTensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [2.28557038])
epoch_id0, batch_id700, lossTensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [2.29461408])
epoch_id0, batch_i

In [10]:
import paddle
import paddle.distributed as dist

def train_multi_gpu(model):
    # 初始化并行环境
    dist.init_parallel_env()
    # 修改2 - 增加paddle.DataParallel封装
    model = paddle.DataParallel(model)
    model.train()
    
    opt = paddle.optimizer.Adam(learning_rate=0.01, parameters=model.parameters())
    EPOCH_NUM = 5
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            #准备数据，变得更加简洁
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)
            #前向计算的过程
            predicts = model(images)
            #计算损失，取一个批次样本损失的平均值
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            #每训练了100批次的数据，打印下当前Loss的情况
            if batch_id % 200 == 0:
                print("epoch: {}, batch: {}, loss is: {}".format(epoch_id, batch_id, avg_loss.numpy()))
            #后向传播，更新参数的过程
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

    #保存模型参数
    paddle.save(model.state_dict(), 'mnist.pdparams')

#创建模型    
model = MNIST()
#启动训练过程
train_multi_gpu(model)

e:\SoftWare\Program\Anaconda\envs\d2l\lib\site-packages\paddle\distributed\parallel.py:173: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  warnings.warn(
e:\SoftWare\Program\Anaconda\envs\d2l\lib\site-packages\paddle\fluid\dygraph\parallel.py:655: UserWarning: The program will return to single-card operation. Please check 1, whether you use spawn or fleetrun to start the program. 2, Whether it is a multi-card program. 3, Is the current environment multi-card.
  warnings.warn("The program will return to single-card operation. "


epoch: 0, batch: 0, loss is: [2.8759856]
epoch: 0, batch: 200, loss is: [0.25942078]
epoch: 0, batch: 400, loss is: [0.1325554]
epoch: 0, batch: 600, loss is: [0.2645532]
epoch: 0, batch: 800, loss is: [0.05263087]
epoch: 1, batch: 0, loss is: [0.09072506]
epoch: 1, batch: 200, loss is: [0.10816649]
epoch: 1, batch: 400, loss is: [0.15476905]
epoch: 1, batch: 600, loss is: [0.16526055]
epoch: 1, batch: 800, loss is: [0.06706055]
epoch: 2, batch: 0, loss is: [0.09617238]
epoch: 2, batch: 200, loss is: [0.00534865]
epoch: 2, batch: 400, loss is: [0.08193104]
epoch: 2, batch: 600, loss is: [0.19643638]
epoch: 2, batch: 800, loss is: [0.15392247]
epoch: 3, batch: 0, loss is: [0.01126755]
epoch: 3, batch: 200, loss is: [0.01784502]
epoch: 3, batch: 400, loss is: [0.13685936]
epoch: 3, batch: 600, loss is: [0.11486576]
epoch: 3, batch: 800, loss is: [0.03811437]
epoch: 4, batch: 0, loss is: [0.10884617]
epoch: 4, batch: 200, loss is: [0.00469687]
epoch: 4, batch: 400, loss is: [0.02162704]
e